# **Sergei Bruchkus, task 2**
Building a model which is capable of rappinng like someone famous
(aka Grime na russkom)...

# ***Here is the second method of solution based mainly on language models***

In [0]:
!pip install lyricsgenius
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.5b1

     |████████████████████████████████| 92kB 4.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 1.4MB 5.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Requirement already up-to-date: dill in /usr/local/lib/python3.6/dist-packages (0.3.1.1)
     |████████████████████████████████| 1.4 MB 4.7 MB/s 
  Created wheel for nltk: filename=nltk-3.5b1-py3-none-any.whl size=1434692 sha256=4290f82083857143034d0ae96618448217f7a509902c1c1bc6909501b8f51973
  Stored in directory: /root/.cache/pip/wheels/d3/86/25/c46dd9a7eced213ff414cc8266b44bb26e61f3f787d36d8dfc
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#importing all the necessary libraries
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import tokenize
from nltk.tokenize import LineTokenizer
from sklearn.model_selection import train_test_split
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer

import lyricsgenius
import pandas as pd
import numpy as np
import nltk
import random

In [0]:

#here we search for all oxxxy songs, we got 88 songs
genius = lyricsgenius.Genius("37_kPKwZgAEV2w8GjOYcuK-2c79H9YUx9PHkN_38aFDJa3Vdxzjveb7YDJ8TSCC1")
genius.remove_section_headers = True # Remove section headers - it came to be really important
artist1 = genius.search_artist("Oxxxymiron",  sort="title")
#print(artist1.songs)

Searching for songs by Oxxxymiron...

Song 1: "16 Bars Acapella"
Song 2: "Afterparty (Demo)"
Song 3: "AI Ogon"
Song 4: "Amplify and simplify (Freestyle)"
Song 5: "CCTV"
Song 6: "Darkside"
Song 7: "Freestyle #1"
Song 8: "Ganz Promo Tune"
Song 9: "Hangover"
Song 10: "HPL"
Song 11: "IMPERIVM"
Song 12: "Intro"
Song 13: "OXXXYMIRON"
Song 14: "Russky Cockney"
Song 15: "Shade 45 Freestyle (Идея)"
Song 16: "Street Freestyle battle"
Song 17: "Ultima Thule"
Song 18: "Unreleased Track"
Song 19: "Unreleased Track 2"
Song 20: "Unreleased Track 3"
Song 21: "Unreleased Track 4"
Song 22: "XXX SHOP"
Song 23: "Башня из слоновой кости (Ivory Tower)"
Song 24: "Биполярочка (Bipolarochka)"
Song 25: "Больше Бена (Bigga Than Ben)"
Song 26: "В бульбуляторе (In the Bong)"
Song 27: "В говне (In Shit)"
Song 28: "В долгий путь (1 раунд 17ib) (On a Long Journey)"
Song 29: "Ветер перемен (2 раунд 17ib) (The Wind of Change)"
Song 30: "Вечный жид (Everlasting Jew)"
Song 31: "Витязи словоблудия (Уховертка) (The Knights

In [0]:
# create txt file with all the lyrics
artist1.save_lyrics(extension='txt')

Lyrics_Oxxxymiron.txt already exists. Overwrite?
(y/n): y
Wrote `Lyrics_Oxxxymiron.txt`


In [0]:



#we open the file and check the format
lyricsoxxxy = open("Lyrics_Oxxxymiron.txt","r")
print (lyricsoxxxy)

<_io.TextIOWrapper name='Lyrics_Oxxxymiron.txt' mode='r' encoding='UTF-8'>


In [0]:
# we prepare tha data to make it appropriate ny using lower case and tokenization
token_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in lyricsoxxxy]

In [0]:
# the same with the first part of this task - here we define a special function, we build a part with 5 lines 
def generate_rap (model, num_lines = 5, num_words_in_line = 10, random_seed = 4773):
  for i in range(num_lines):
    content = []
    for token in model.generate(num_words_in_line, random_seed=random_seed+i):
      if token == '<s>':
        continue
      if token == '</s>':
        break #the symbol for the end
      content.append(token)
    content = TreebankWordDetokenizer().detokenize(content)
    if bool(content):
      i = 0
      while not content[i].isalpha():
          i += 1
          if i > len(content)-1:
            break
      print(content[i:].capitalize())

In [0]:
#split the data on training and test samples
train, test = train_test_split(token_text, test_size=0.2, random_state=4773)

In [0]:
#we apply three different models

#firstly, we apply models based on unigramms

model1 = MLE(1) #maximum likelyhood estmation, we evaluate the probability of the next word based on the prior
train_data, padded_sents = padded_everygram_pipeline(1, train)
model1.fit(train_data, padded_sents)

model2 = Laplace(1) #the one we had a look atm a methof useful in cases when we have zero probability of meeting some words
train_data1, padded_sents1 = padded_everygram_pipeline(1, train)
model2.fit(train_data1, padded_sents1)

model3 = KneserNeyInterpolated(1) #Interpolated version of Kneser-Ney smoothing
train_data2, padded_sents2 = padded_everygram_pipeline(1, train)
model3.fit(train_data2, padded_sents2)




In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer


In [0]:
generate_rap(model1, random_seed = 4773)
#the fact of combining russian and english lyrics is okay, generally

В! стуле но меня так ,?
Оплеухи, здесь скачаешь"порох друзья ведь что хоккеист
Измени к даже your, уверены и закончил )
Доим похлеще свою витает и применяли быстро это все тоже
Но на карикатурах улицам какого нет голубого . — машина


In [0]:
generate_rap(model2, random_seed = 4773)

Верну! сто осадки мог схваткам, m
Переварю, и сколько (привкусом жук всего чистую франциск
I калининграде клитор дом бег . у каждый и *
Ждёт причитаешь себя выведи из пророс вдаль экосистем глазах тихой
Она нарочно крики уж кочевал облысел дело download қотақбас мне


In [0]:
generate_rap(model3, random_seed = 4773)

Russia белой делегат're сраный поехали ниц стеклом ау быстрый
Порою over кукол словаря aмур пьянство кладу друзьями хаджеводокгхида унесу
Бугор марк месте иносказательно выбито бабулек трэшак лучше крекеры buddy
Кашель разбегу симоны ересиарх лев распадаются девкам цепь жужжит таляляляля
Подписать островной мораль тьме моисея победы золотая бонзам эдика низы


In [0]:
test_data1,_ = padded_everygram_pipeline(1, test)
#to test the models on separated in the beginning test sample 

In [0]:
test_gen1 = []

for line in test_data1:
  test_gen1.extend(line)


In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model1.perplexity(test_gen1)))
print('Perplexity for unigram Laplace model is: {}'.format(model2.perplexity(test_gen1)))
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model3.perplexity(test_gen1)))


Perplexity for unigram MLE model is: inf
Perplexity for unigram Laplace model is: 1625.375745453456
Perplexity for unigram KneserNeyInterpolated model is: 9939.999999987811


If we judge by perplexity and compare the models, we can say that the first one show a way worse results compared to Laplace and KnesserNeyInterpolated (since perplexity is innfinity)
Also, here, when we use unigrams, Laplace smoothinng is more effective (more probable lyrics on test sample) than KneserNeyInterpolated

In [0]:
#the same using bigrams, therefore we expect to get a something more likely to be a real way of words sequences by oxxxymiron
model4 = MLE(2)
train_data3, padded_sents3 = padded_everygram_pipeline(2, train)
model4.fit(train_data3, padded_sents3)

model5 = KneserNeyInterpolated(2)
train_data4, padded_sents4 = padded_everygram_pipeline(2, train)
model5.fit(train_data4, padded_sents4)

model6 = Laplace(2)
train_data5, padded_sents5 = padded_everygram_pipeline(2, train)
model6.fit(train_data5, padded_sents5)

In [0]:
generate_rap(model4, random_seed = 4773)
#hear i iunderstood that should have removed headers annd now i start from the beginning

Брат ,
На базаре продаётся предсказание кассандры
Бабана пишут такие modern talking (а я жопу рвать
Мне не матерится, мимо икон


In [0]:
generate_rap(model5, random_seed = 4773)

Runnin' you'll tec-9 when i ’ m a
Порочишь её в сердце, само собою, экскюз ми
Бу ...
Кацапов, тебе восемь лет пролетело, я жопу рвать
Подписан на меня так легко-оу уо-о


It's pretty interesting that the previous block starts with english word and as a result, the whole string of the lyrics is English (since it's more likely than English word is followed by another english word).
Also I have no idea why phrase "zhopu rvat'" appeared twice here.
Also, for some reason we got 4 lines running MLE model and Laplace. Probably it's due to tehnical problems while running code and such a long time needed.

In [0]:
generate_rap(model6, random_seed = 4773)

Больше дел, тощий стебель
Нелепые лохи недовольны всегда (пау!
Венерических болезней
Напас!




```
For some reason, here we have pretty strange lyrics here...```



In [0]:
test_data2,_ = padded_everygram_pipeline(2, test)


In [0]:
test_gen2 = []

for line in test_data2:
  test_gen2.extend(line)


In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model4.perplexity(test_gen2)))
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model5.perplexity(test_gen2)))
print('Perplexity for unigram Laplace model is: {}'.format(model6.perplexity(test_gen2)))

Perplexity for unigram MLE model is: inf
Perplexity for unigram KneserNeyInterpolated model is: 4379.608163733989
Perplexity for unigram Laplace model is: 1597.378902532775


Again, judging by perplexity, MLE model performs very bad and there's almost zero chance to get some likely lyrics when we test the data.
Laplace performance is still better than one my KnesserNeyInterpolation, even with bigrams.

In [0]:
#trigrams implementation, now the texts should be even more coherent annd logic
model7 = MLE(3)
train_data6, padded_sents6 = padded_everygram_pipeline(3, train)
model7.fit(train_data6, padded_sents6)

model8 = Laplace(3)
train_data7, padded_sents7 = padded_everygram_pipeline(3, train)
model8.fit(train_data7, padded_sents7)

model9 = KneserNeyInterpolated(3)
train_data8, padded_sents8 = padded_everygram_pipeline(3, train)
model9.fit(train_data8, padded_sents8)

In [0]:
generate_rap(model7, random_seed = 4773)

Брат, как темно
Котом и некрономикон
Когда я ещё был не в тот век ,
Йе )


we even have some rhyme here

In [0]:
generate_rap(model8, random_seed = 4773)

Больше дев, не проводи большой обыск
Моей головой круглый неон — это кал, одна пуля
Gimme some money now!
Ложь не яд


In [0]:
generate_rap(model9, random_seed = 4773)

Runnin' you can ’ t kill me, i
Порочишь её криками``saint-p, man ''
Бу...пере пуроста
Кацапов, хачей и для русаков
Подписан на рип . клеил чик, делал биз


The textes we get have a lot of common with real texts of this rapper, since with use the probabilities of having three words in a row.

In [0]:
test_data3,_ = padded_everygram_pipeline(3, test)

In [0]:
test_gen3 = []

for line in test_data3:
  test_gen3.extend(line)

In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model7.perplexity(test_gen3)))
print('Perplexity for unigram Laplace model is: {}'.format(model8.perplexity(test_gen3)))
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model9.perplexity(test_gen3)))

Perplexity for unigram MLE model is: inf
Perplexity for unigram Laplace model is: 1170.6233617271685
Perplexity for unigram KneserNeyInterpolated model is: 1849.517337207427


Laplace is still the best option, even though KneserNeyInterpolation is pretty close to it.

**Here we'll try to add some other russian rap artist to this collection to ennlarge the vocabulary.** 

We would decide on the artist based on some vocabulary similarities with oxxxymiron (from some personal experience)

In [0]:
genius = lyricsgenius.Genius("37_kPKwZgAEV2w8GjOYcuK-2c79H9YUx9PHkN_38aFDJa3Vdxzjveb7YDJ8TSCC1")
genius.remove_section_headers = True # Remove section headers
#add other artists
Hasky = genius.search_artist("Хаски", max_songs=50, sort="title")
Markul = genius.search_artist("Markul", max_songs=50, sort="title")
Johny = genius.search_artist("Johnyboy", max_songs=50, sort="title")
artist1 = genius.search_artist("Oxxxymiron", max_songs=50, sort="title")

Searching for songs by Хаски...

Changing artist name to 'Хаски (Husky)'
Song 1: "Ай (Ai)"
Song 2: "Аллилуйя (Hallelujah)"
Song 3: "Бит шатает голову (Beat Shakes The Head)"
Song 4: "Бэнг Бэнг (Bang Bang)"
Song 5: "Все псы попадают в рай (All Dogs Go To Heaven)"
Song 6: "Голубка (Dove)"
Song 7: "Господин Собака (Mr. Dog)"
Song 8: "Детка-Голливуд (Baby-Hollywood)"
Song 9: "Дурачок (Fool)"
Song 10: "Животворящий Флоу (Vital Flow)"
Song 11: "Заново (Once Again)"
Song 12: "Иисус (Jesus)"
Song 13: "Иуда (Judas)"
Song 14: "Карлсон (Carlson)"
Song 15: "Колдунья(Witch)"
Song 16: "Космолёт (Space Shuttle)"
Song 17: "Крот ’17 (Mole 17)"
Song 18: "Крот (Mole)"
Song 19: "Люцифер (Lucifer)"
Song 20: "Мармелад (Marmalade)"
Song 21: "Моим людям (To My People)"
Song 22: "Мультики (Cartoons)"
Song 23: "Недо(бо)-человек"
Song 24: "Отопление (Heating)"
Song 25: "Панелька (Panel House)"
Song 26: "Пироман (Pyromaniac)"
Song 27: "Пироман ’17 (Pyromaniac)"
Song 28: "Поэма о Родине (Poem About Motherland)"
So

In [0]:
Hasky.save_lyrics(extension='txt')
artist1.save_lyrics(extension='txt')
Markul.save_lyrics(extension='txt')
Johny.save_lyrics(extension='txt')
#like we did with oxxxymiron lyrics

Wrote `Lyrics_ХаскиHusky.txt`
Wrote `Lyrics_Oxxxymiron.txt`
Wrote `Lyrics_MARKUL.txt`
Wrote `Lyrics_Johnyboy.txt`


In [0]:
Hasky_lyr= open("Lyrics_ХаскиHusky.txt","r")
Mark_lyr = open("Lyrics_MARKUL.txt","r")
Johny_lyr = open("Lyrics_Johnyboy.txt","r")
lyricsoxxxy = open("Lyrics_Oxxxymiron.txt","r")

In [0]:

Hasky_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Hasky_lyr]
Mark_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Mark_lyr]
Johny_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Johny_lyr]
token_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in lyricsoxxxy]

In [0]:
lyricst = [] #for storing the whole corpus of lyrics
lyricst.extend(token_text)
lyricst.extend(Hasky_tokenized)
lyricst.extend(Mark_tokenized)
lyricst.extend(Johny_tokenized)
print(len(lyricst))

11695


impressive...

In [0]:
train, test = train_test_split(lyricst, test_size=0.2, random_state=1212)

In [0]:
#we use the same algorithm with 3 models, and uni-, bi- and thigrams
#firstly, unnigrms
model1 = MLE(1)
train_data, padded_sents = padded_everygram_pipeline(1, train)
model1.fit(train_data, padded_sents)

model2 = Laplace(1)
train_data1, padded_sents1 = padded_everygram_pipeline(1, train)
model2.fit(train_data1, padded_sents1)

model3 = KneserNeyInterpolated(1)
train_data2, padded_sents2 = padded_everygram_pipeline(1, train)
model3.fit(train_data2, padded_sents2)

In [0]:
generate_rap(model1, random_seed = 1212)

Сигануть вся вот зашторишь пути люди много грехов и тут
Люлей кто boyfriend не заметно люди могла, вместе все
Облик, огней жизни но котором называют сказать проснешься
Из со закрывают говоря пола ты мотиватор и на гниёт
Мимо, , мы, баронов фанфарон о-о нам спасти


In [0]:
generate_rap(model2, random_seed = 1212)

Сирано голоса выцветшей и разве металла муках джо интернет три
Мечтал лоик wild но и метит мы, втирай гид
Отвратительно, отстой захочешь окно летим не сколько пуст
Как собака и девки потому ты на инстинктов начинают дворец
Моисеев, , набито, бродяг уродливые от не сохраняет


In [0]:
generate_rap(model3, random_seed = 1212)

Скрываюсь запрете зажить кэш риске нету огригами иконою лучшим терпел
Нефть насквозь возрасту пластырь кудахтали неужели одна американская забойным заливаю
Эскалатор понят беду порвём костей показан налетай париже следует ребята
Махинатор собирательный крыша зря пульта топы орать лучшие падать зомби
Обитает swag ае острый time голытьба уважения поможешь пассажирский сопли


In [0]:
test_data1,_ = padded_everygram_pipeline(1, test)


In [0]:
test_gen1 = []

for line in test_data1:
  test_gen1.extend(line)


In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model1.perplexity(test_gen1)))
print('Perplexity for unigram Laplace model is: {}'.format(model2.perplexity(test_gen1)))
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model3.perplexity(test_gen1)))


Perplexity for unigram MLE model is: inf
Perplexity for unigram Laplace model is: 1706.364981704953
Perplexity for unigram KneserNeyInterpolated model is: 15534.99999996799


In [0]:
model4 = MLE(2)
train_data3, padded_sents3 = padded_everygram_pipeline(2, train)
model4.fit(train_data3, padded_sents3)

model5 = KneserNeyInterpolated(2)
train_data4, padded_sents4 = padded_everygram_pipeline(2, train)
model5.fit(train_data4, padded_sents4)

model6 = Laplace(2)
train_data5, padded_sents5 = padded_everygram_pipeline(2, train)
model6.fit(train_data5, padded_sents5)

In [0]:
generate_rap(model4, random_seed = 1212)

Прогрызает вовне
Желе
Спазм
Воу
Звездам


It is noticeable that we got one word in every string. Probably we can make a conclusion, that sometimes these rappers use so rare words, that probability of contnuation is lower than the probability this word is followed by nothing.

In [0]:
generate_rap(model5, random_seed = 1212)

Скрывать всё ещё не пушистый
Нефтегаз )
Эскалатор
Мафона и мне всё равно рядом ложусь
Обитаем там, но


The same as in the previous model: the word нефтегаз or эскалатор are not likely to be followed by anything

In [0]:
generate_rap(model6, random_seed = 1212)

Развод, зверь неведомый
И не всегда видит бедуин
Ярко тут
Два скупленных тейпа, разъеби чувака в мире существует вандализм
Искусства нико пиросмани


In [0]:
test_data2,_ = padded_everygram_pipeline(2, test)


In [0]:
test_gen2 = []

for line in test_data2:
  test_gen2.extend(line)

In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model4.perplexity(test_gen2)))
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model5.perplexity(test_gen2)))
print('Perplexity for unigram Laplace model is: {}'.format(model6.perplexity(test_gen2)))

Perplexity for unigram MLE model is: inf
Perplexity for unigram KneserNeyInterpolated model is: 4299.172680505865
Perplexity for unigram Laplace model is: 1580.3979909430316


Results are prette the same with when we used only Oxxxymiron's texts

In [0]:
#trigrams
model7 = MLE(3)
train_data6, padded_sents6 = padded_everygram_pipeline(3, train)
model7.fit(train_data6, padded_sents6)

model8 = Laplace(3)
train_data7, padded_sents7 = padded_everygram_pipeline(3, train)
model8.fit(train_data7, padded_sents7)

model9 = KneserNeyInterpolated(3)
train_data8, padded_sents8 = padded_everygram_pipeline(3, train)
model9.fit(train_data8, padded_sents8)

In [0]:
generate_rap(model7, random_seed = 1212)

Памяти у всего мира все нигеры поднимают свои руки вверх
В никуда
Я пишу, это день плохого парня (парня )
Оба разобьемся о скалы
Видал


In [0]:
generate_rap(model8, random_seed = 1212)

Полемики
Голова полна дерьма - черепушка, как блюз
Я поздравляю, ты земляк скита и флига, ведь
Блядь »
Демонстрирует норов, улыбается


In [0]:
generate_rap(model9, random_seed = 1212)

Скрывать всё, то хит
Нефтегаз )
Эскалатор
Мафона и шахматиста?
Обитаем там, где берега кисельные


In [0]:
test_data3,_ = padded_everygram_pipeline(3, test)


In [0]:
test_gen3 = []

for line in test_data3:
  test_gen3.extend(line)

In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model7.perplexity(test_gen3)))
print('Perplexity for unigram Laplace model is: {}'.format(model8.perplexity(test_gen3)))
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model9.perplexity(test_gen3)))

Perplexity for unigram MLE model is: inf
Perplexity for unigram Laplace model is: 1115.3910780171748
Perplexity for unigram KneserNeyInterpolated model is: 1460.751565342289


We also can say that when we come to using trigrams, the performance of KneserNeyImplementation is getting closer to one of Laplace smoothing.

Generally, the task was really innteresting as a chance to to see how different models behave on something cool not boring (like rap songs)
I would say that it takes very very long to run some models, and very very long to wait for the NN algorithm (taking into consideration that we have to have quite a lot of epochs to train our model well).
Even though we can see the lowest perplexity with the Laplace smoothing model (higher performance on the test sample), we should also say, that, firstly, perplexity of KnesserNeyInterpolation becomes lower when we move to longer word orders (from unnigrams to trigrams). And, secondly, perplexity of MLE model is always (in our cases) infinity, but it is executed a way faster and I would't say that the generated rap is any worse than one generated by other models.
Finally, it would be interesting to add even more russian rap artist to the list to see the results. But it would take a lot of time and capacities.